In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch
import torch
print(torch.__version__)

!pip install pandas

Change torch.__version__ of following installation command.


In [ ]:
!pip install torch-geometric torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html

Preprocess data

In [ ]:
import os
os.listdir('./drive/MyDrive/DSCI558/')

['HW1.gdoc',
 'DSCI 558 Final Project Proposal.gdoc',
 'DSCI558_Project_Proposal.gslides',
 'dsci558quiz',
 'clicks.dat',
 'transaction.dat']

In [ ]:
import pandas as pd

# Load files. (You should change file name according to your storage path)
clicks_df = pd.read_csv('./drive/MyDrive/DSCI558/clicks.dat', header=None)
clicks_df.columns=['session_id','timestamp','item_id','category']
clicks_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,214536502,0
1,1,2014-04-07T10:54:09.868Z,214536500,0
2,1,2014-04-07T10:54:46.998Z,214536506,0
3,1,2014-04-07T10:57:00.306Z,214577561,0
4,2,2014-04-07T13:56:37.614Z,214662742,0


In [ ]:
buy_df = pd.read_csv('./drive/MyDrive/DSCI558/transaction.dat', header=None)
buy_df.columns=['session_id','timestamp','item_id','price','quantity']
buy_df.head()

,session_id,timestamp,item_id,price,quantity
0,420374,2014-04-06T18:44:58.314Z,214537888,12462,1
1,420374,2014-04-06T18:44:58.325Z,214537850,10471,1
2,281626,2014-04-06T09:40:13.032Z,214535653,1883,1
3,420368,2014-04-04T06:13:28.848Z,214530572,6073,1
4,420368,2014-04-04T06:13:28.858Z,214835025,2617,1


Sample data: Randomly sample 100K sessions (due to huge file size)

In [ ]:
import numpy as np
sampled_session_id = np.random.choice(clicks_df.session_id.unique(), 100000, replace=False)
clicks_df = clicks_df.loc[clicks_df.session_id.isin(sampled_session_id)]
clicks_df.nunique()

session_id    100000
timestamp     358852
item_id        20629
category         123
dtype: int64

In [ ]:
clicks_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 358863 entries, 616 to 33003936
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   session_id  358863 non-null  int64 
 1   timestamp   358863 non-null  object
 2   item_id     358863 non-null  int64 
 3   category    358863 non-null  object
dtypes: int64(2), object(2)
memory usage: 13.7+ MB


Encode labels with values between 0 to (num_classes - 1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
item_encoder = LabelEncoder()
category_encoder = LabelEncoder()
clicks_df['item_id']=item_encoder.fit_transform(clicks_df.item_id)
clicks_df['category']=category_encoder.fit_transform(clicks_df.category.apply(str))
clicks_df.head()

,session_id,timestamp,item_id,category
616,194,2014-04-04T14:29:16.071Z,14527,0
617,194,2014-04-04T14:29:47.845Z,14527,0
618,194,2014-04-04T14:31:10.979Z,14527,0
619,194,2014-04-04T14:36:37.840Z,14527,0
676,238,2014-04-04T19:01:27.919Z,12787,0


In [ ]:
buy_df=buy_df.loc[buy_df.session_id.isin(clicks_df.session_id)]
buy_df['item_id']=item_encoder.transform(buy_df.item_id)
buy_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,session_id,timestamp,item_id,price,quantity
106,281747,2014-04-06T10:32:23.948Z,14739,523,2
107,281747,2014-04-06T10:32:23.984Z,14739,523,1
108,281747,2014-04-06T10:32:24.000Z,3810,3141,1
109,281747,2014-04-06T10:32:24.020Z,14692,313,1
110,281747,2014-04-06T10:32:24.036Z,12706,7852,1


Create session_id - Item dictionary (Who bought which item?)

In [ ]:
buy_item_dict = dict(buy_df.groupby('session_id')['item_id'].apply(list))
buy_item_dict

{1821: [13020],
 2547: [16976],
 3608: [14740, 14481],
 3778: [4073],
 4988: [16729],
 8337: [14413, 14443, 9842, 6491],
 9277: [12787],
 9307: [14697, 5618, 9978, 16591, 14701],
 14783: [14483, 14479, 14481, 14484, 14754, 374],
 15609: [12752],
 15709: [14713, 14739, 14739, 14815, 14714, 14715],
 18467: [15365],
 20178: [14691, 14480],
 23184: [14483, 14483],
 23273: [14989],
 26283: [15436],
 28063: [12115],
 29318: [4674],
 32599: [14207],
 32722: [14754, 17072],
 32913: [14339],
 37077: [12748],
 38844: [16729],
 39163: [14483, 14479, 14484, 14481, 14739],
 39304: [5686],
 41348: [9503, 9503],
 42553: [16729],
 43674: [14288],
 44206: [12754],
 44451: [16177, 5281],
 45068: [12785],
 48108: [12750, 12785],
 48849: [14733, 14733, 14523, 14733],
 49158: [15455],
 49981: [12785, 12749],
 51369: [15456],
 52952: [10876],
 54262: [9557, 14914],
 55433: [12771, 12770],
 59143: [1098],
 61886: [11992],
 62877: [13448],
 63126: [14714, 14488],
 64198: [14343, 14331],
 65172: [12771],
 7164

Define custom dataset class for train GNN
Treat each item in a session as node → All items in the same session form a graph
Items will be connected by edge.


In [ ]:
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

from tqdm import tqdm
class CustomDataset(InMemoryDataset):
   def __init__(self, root, transform=None, pre_transform=None):
       super(CustomDataset, self).__init__(root, transform, pre_transform)
       self.data, self.slices = torch.load(self.processed_paths[0])
 
   @property
   def raw_file_names(self):
       return []
 
   @property
   def processed_file_names(self):
       return ['save.dataset']
 
   def download(self):
       pass
  
   def process(self):
      
       data_list = []
       # process by session_id
       grouped = clicks_df.groupby('session_id')
       for session_id, group in tqdm(grouped):
           le = LabelEncoder()
           sess_item_id = le.fit_transform(group.item_id)
           group = group.reset_index(drop=True)
           group['sess_item_id'] = sess_item_id
           node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id','category']].sort_values('sess_item_id')[['item_id','category']].drop_duplicates().values
           node_features = torch.LongTensor(node_features).unsqueeze(1)
           target_nodes = group.sess_item_id.values[1:]
           source_nodes = group.sess_item_id.values[:-1]
           edge_index = torch.tensor([source_nodes,
                                  target_nodes], dtype=torch.long)
           x = node_features
 
           if session_id in buy_item_dict:
               positive_indices = le.transform(buy_item_dict[session_id])
               label = np.zeros(len(node_features))
               label[positive_indices] = 1
           else:
               label = [0] * len(node_features)
 
           y = torch.FloatTensor(label)
           data = Data(x=x, edge_index=edge_index, y=y)
 
           data_list.append(data)
      
       data, slices = self.collate(data_list)
       torch.save((data, slices), self.processed_paths[0])
 
dataset = CustomDataset('../')


Processing...
100%|██████████| 100000/100000 [09:13<00:00, 180.62it/s]
Done!


In [ ]:
len(dataset)

100000

In [ ]:
dataset.num_classes

2

In [ ]:
dataset.num_node_features

2

Model

In [42]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.nn import global_mean_pool

num_items = clicks_df.item_id.max() +1
num_categories = clicks_df.category.max()+1
embed_dim = 128

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.item_embedding = torch.nn.Embedding(num_embeddings=num_items, embedding_dim=embed_dim)
        self.category_embedding = torch.nn.Embedding(num_embeddings=num_categories, embedding_dim=embed_dim)        
        self.conv1 = SAGEConv(embed_dim * 2, embed_dim, aggr='max') # You can use other graph convolution layer
        self.conv2 = SAGEConv(embed_dim, 64, aggr='max')
        self.conv3 = SAGEConv(64, 16, aggr='max')
        self.lin1 = Linear(16, 1) # You can multiple linear layers
        self.act1 = torch.nn.ReLU()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch  
        item_id = x[:,:,0]
        category = x[:,:,1]

        # Use item embedding & category embedding to solve the problem.
        emb_item = self.item_embedding(item_id).squeeze(1)
        emb_category = self.category_embedding(category).squeeze(1)
        x = torch.cat([emb_item, emb_category], dim=1)

        ### You can add layers or alter the model structure. See geometric documents which layer or model you can use.
        x = self.conv1(x, edge_index) # x.shape = (number of edges in the batch of graphs, representation dimension)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)

        #x = global_mean_pool(x, batch)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        ### Prediction score for each edge, range 0~1
        x = torch.sigmoid(self.lin1(x)).squeeze(1)
        
        return x  

Dataloader

In [ ]:
### add train, val, test loader.
from torch_geometric.loader import DataLoader

train_dataset = dataset[:80000]
val_dataset = dataset[80000:90000]
test_dataset = dataset[90000:]

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

Train

In [ ]:
def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)

        label = data.y.to(device)
        loss = torch.nn.BCELoss()(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

Val

In [ ]:
def val(loader):
    model.eval()

    loss_all = 0
    for data in loader:
        data = data.to(device)
        #optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = torch.nn.BCELoss()(output, label)
        #loss.backward()
        loss_all += data.num_graphs * loss.item()
        #optimizer.step()
    return loss_all / len(val_dataset)

Test

In [ ]:
from sklearn.metrics import roc_auc_score
def test(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)

    return roc_auc_score(labels, predictions)

In [43]:
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
#data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()

number_of_epochs = 15 # You can change.

tmp = math.inf
best_model = model
best_epoch = 0
for epoch in range(number_of_epochs):
    train_loss = train()
    val_loss = val(val_loader)

    # Choose the lowest validation loss checkpoint (you can implement early stopping as well)
    if val_loss < tmp:
      tmp = val_loss
      best_model = model 
      best_epoch = epoch
    print(val_loss)   
print(f'Best_epoch: {best_epoch}, min_val_loss: {tmp}')

# Load the lowest validation loss checkpoint and check the performance.
model = best_model
test_acc = test(test_loader)
print('Performance: ', test_acc)

0.18334408979415895
0.18486498938798904
0.18212215064167978
0.18440466226935387
0.18490551503300667
0.18457373180389405
0.18374463242292405
0.1842633683681488
0.18402901933193208
0.18695995478630065
0.18329678843021394
0.18575321979522705
0.18447983254194258
0.19737278044223786
0.18452064890265465
Best_epoch: 2, min_val_loss: 0.18212215064167978
Performance:  0.6274697813577657
